In [ ]:
 # This cell is tagged with "parameters" and used by papermill
# the outpath will be overridden to keep the name backward compatible if it is from auto-processing

# default settings
inpath = "" # type: str
outpath = "" # type: str
process_configfile = "/dls_sw/i14/ops/processing/auto/pyfai_live.yaml" # type: str
calibration_path = "" # type: str
mask_path = "" # type: str
flat_path = "" # type: str
background_type = "" # type: str
polynomial_order = 0 # type: int
auto_processing = False # type: bool

## Import all the necessary packages
(module load python/epsic3.10)

In [ ]:
from importlib.metadata import version as imlib_ver
import subprocess
import yaml

import h5py
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from pyFAI.detectors import Detector
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.ext.morphology import binary_dilation
from pyFAI.utils import crc32
from pyFAI import units
print(f"pyFAI version: {imlib_ver('pyfai')}")

In [ ]:
from i14_utility.xrd.integrate_direct import ExcaliburXRDIntegration

## Load all the necessary configurations

In [ ]:
# get processing configuration
if not process_configfile:
    # points to the base pyFAI configuration file
    process_configfile = "/dls_sw/i14/ops/processing/auto/pyfai_live.yaml"

In [ ]:
# typing out all here to ensure it doesn't depend on the base pyFAI configuration file
dataset_key_default = "/entry/excalibur_addetector/data"
dataset_path_default = "/entry/excalibur_addetector"

mask_default = {"mask_path": "/dls/i14/data/2024/cm37259-1/processing/xrd_calibration/mask_edge.nxs",
                "mask_key": "/entry/mask/mask",
                "dilate_mask": True,
               }

flat_default = {"flat_path": "/dls/i14/data/2024/cm37259-1/processing/xrd_calibration/ffcoeffs_Zr_fixed.hdf",
                "flat_key": "/entry/data",
               }

do_regrid_default = True

xrd1d_default = {"perform": True,
                 "npt_rad": 1024,
                 "unit": "q_A^-1",
                 "toSave": True,
                 "correctSolidAngle": True,
                 "polarization_factor": 0.95,
                 "radial_min": None,
                 "radial_max": None,
                 "azimuthal_min": None,
                 "azimuthal_max": None,
                 "safe": False,
                }

# The navigation dimension can be cropped. The first entry is the starting pixel
# (start from 0) and the second entry is the last pixel. For example, if only
# the first 10 pixels in the x coordinate is needed, set crop/x/start to
# 0 or null and crop/x/end to 10.
# If no cropping is needed, just leave them as null
crop_default = {"x": {"start": None, "end": None},
                "y": {"start": None, "end": None},
               }

background_removal_default = {"remove_background": True, # perform background removal or not
                              "background_type": "Polynomial", # can be Doniach, Gaussian, Lorentzian, Offset, Polynomial, PowerLaw, Exponential, SkewNormal, SplitVoigt, Voigt or Empty
                              "polynomial_order": 7, # only active if Polynomial is chosen
                              "background_file": None, # only active if Empty is chosen
                              "bg_start": None, # the range of the radial axis that the background model is to be fitted
                              "bg_end": None, # set to null for full range
                              "smoothing_weight": 0, # smoothing weight for total variation denoising, 0 for no smoothing
                             }

In [ ]:
try:
    f = open(process_configfile)
except FileNotFoundError:
    print(f"Configuration file {process_configfile} is not found. Default configurations will be used.")

    process_config = {"calibration_path": "",
                      "dataset_key": dataset_key_default,
                      "dataset_path": dataset_path_default,
                      "mask": mask_default,
                      "flat": flat_default,
                      "do_regrid": do_regrid_default,
                      "xrd1d": xrd1d_default,
                      "crop": crop_default,
                      "background_removal": background_removal_default,
                     }
else:
    process_config = yaml.load(f.read(), Loader=yaml.FullLoader)
    f.close()

In [ ]:
if not calibration_path:
    # calibration_path was not passed as a parameter, get it from the configuration file
    calibration_path = process_config.get("calibration_path", "")
    
dataset_path = process_config.get("dataset_path", dataset_path_default)
do_regrid = process_config.get("do_regrid", do_regrid_default)

In [ ]:
mask_config = process_config.get("mask", mask_default)

if not mask_path:
    # mask_path was not passed as a parameter, get it from the configuration file
    mask_path = mask_config.get("mask_path", mask_default["mask_path"])
    
mask_key = mask_config.get("mask_key", mask_default["mask_key"])
dilate_mask = mask_config.get("dilate_mask", mask_default["dilate_mask"])

In [ ]:
flat_config = process_config.get("flat", flat_default)

if not flat_path:
    # flat_path was not passed as a parameter, get it from the configuration file
    flat_path = flat_config.get("flat_path", flat_default["flat_path"])
    
flat_key = flat_config.get("flat_key", flat_default["flat_key"])

In [ ]:
print(f"{calibration_path = }")
print(f"{dataset_path = }")
print(f"{mask_path = }")
print(f"{mask_key = }")
print(f"{dilate_mask = }")
print(f"{flat_path = }")
print(f"{flat_key = }")

In [ ]:
xrd1d_config = process_config.get("xrd1d", xrd1d_default)

npt_rad = xrd1d_config.get("npt_rad", xrd1d_default["npt_rad"])
unit = xrd1d_config.get("unit", xrd1d_default["unit"])
toSave = xrd1d_config.get("toSave", xrd1d_default["toSave"])
correctSolidAngle = xrd1d_config.get("correctSolidAngle", xrd1d_default["correctSolidAngle"])
polarization_factor = xrd1d_config.get("polarization_factor", xrd1d_default["polarization_factor"])
radial_min = xrd1d_config.get("radial_min", xrd1d_default["radial_min"])
radial_max = xrd1d_config.get("radial_max", xrd1d_default["radial_max"])
azimuthal_min = xrd1d_config.get("azimuthal_min", xrd1d_default["azimuthal_min"])
azimuthal_max = xrd1d_config.get("azimuthal_max", xrd1d_default["azimuthal_max"])
safe = xrd1d_config.get("safe", xrd1d_default["safe"])

In [ ]:
crop_config = process_config.get("crop", crop_default)
crop_x_config = crop_config.get("x", crop_default["x"])
crop_y_config = crop_config.get("y", crop_default["y"])

crop_x_start = crop_x_config.get("start", crop_default["x"]["start"])
crop_x_end = crop_x_config.get("end", crop_default["x"]["end"])
crop_y_start = crop_y_config.get("start", crop_default["y"]["start"])
crop_y_end = crop_y_config.get("end", crop_default["y"]["end"])
crop_x = slice(crop_x_start, crop_x_end)  #start, end (None means default)
crop_y = slice(crop_y_start, crop_y_end)  #start, end (None means default)

In [ ]:
background_removal_config = process_config.get("background_removal", background_removal_default)

remove_background = background_removal_config.get("remove_background", background_removal_default["remove_background"])
if not background_type:
    # background_type was not passed as a parameter, get it from the configuration file
    background_type = background_removal_config.get("background_type", background_removal_default["background_type"])
if not polynomial_order:
    # polynomial_order was not passed as a parameter, get it from the configuration file
    polynomial_order = background_removal_config.get("polynomial_order", background_removal_default["polynomial_order"])
bg_start = background_removal_config.get("bg_start", background_removal_default["bg_start"])
bg_end = background_removal_config.get("bg_end", background_removal_default["bg_end"])
smoothing_weight = background_removal_config.get("smoothing_weight", background_removal_default["smoothing_weight"])

In [ ]:
print(f"{npt_rad = }")
print(f"{unit = }")
print(f"{toSave = }")
print(f"{correctSolidAngle = }")
print(f"{polarization_factor = }")
print(f"{radial_min = }")
print(f"{radial_max = }")
print(f"{azimuthal_min = }")
print(f"{azimuthal_max = }")
print(f"{safe = }")
print(f"{crop_x = }")
print(f"{crop_y = }")
print()
print(f"{remove_background = }")
if remove_background:
    print(f"{background_type = }")
    print(f"{polynomial_order = }")
    print(f"{bg_start = }")
    print(f"{bg_end = }")
    print(f"{smoothing_weight = }")

In [ ]:
if not calibration_path:
    msg = f"Invalid calibration path {calibration_path}, please check the path of the calibration"
    raise ValueError(msg)

## Check if GPU is available and select appropriate integration implementation 
Check if any (NVIDIA) GPU is available

In [ ]:
try:
    ngpu = str(subprocess.check_output(["nvidia-smi", "-L"])).count("UUID")
except:
    ngpu = 0

# select integration implementation: (splitting, algorithm, implementation)
# select integration method based on availibility of GPU
if ngpu:
    method = ("bbox", "csr", "opencl")
else:
    method = ("bbox", "lut", "cython")
    
print(f"Number of GPU: {ngpu}")
print(f"Integration implementation: {method}")

In [ ]:
if unit == "2th_deg":
    tth = True
    angstrom = False
    unit_pyfai = "q_nm^-1"
elif unit == "q_A^-1":
    # setting angstrom directly won't work?
    tth = False
    angstrom = True
    unit_pyfai = "q_nm^-1"
    
    if radial_min is not None:
        radial_min *= 10
    if radial_max is not None:
        radial_max *= 10
else:
    tth = False
    angstrom = False
    unit_pyfai = unit

## Load the mask

In [ ]:
with h5py.File(mask_path, "r") as fm:
    try:
        dset_mask = fm[mask_key]
    except KeyError:
        msg = (f"The mask file {mask_path} does not contain the dataset {mask_key}. "
                "Check the file and pass the correct key for the mask.")
        raise KeyError(msg) from None
    else:
        mask = dset_mask[()]
        
if dilate_mask:
    mask = binary_dilation(mask, radius=1.0)    

## Load the calibration

In [ ]:
with h5py.File(calibration_path, "r") as f:
    px_x = f["/entry1/instrument/detector/detector_module/fast_pixel_direction"][...]
    px_y = f["/entry1/instrument/detector/detector_module/slow_pixel_direction"][...]
    ydim = f["/entry1/instrument/detector/detector_module/data_size"][...][0]
    xdim = f["/entry1/instrument/detector/detector_module/data_size"][...][1]

    bc_x = f["/entry1/instrument/detector/beam_center_x"][...]
    bc_y = f["/entry1/instrument/detector/beam_center_y"][...]
    distance = f["/entry1/instrument/detector/distance"][...]

    wavelength = f["/entry1/calibration_sample/beam/incident_wavelength"][...]

    eub = f["/entry1/instrument/detector/transformations/euler_b"][...]
    euc = f["/entry1/instrument/detector/transformations/euler_c"][...]

if not np.isclose(px_x, 0.055) or not np.isclose(px_y, 0.055):
    print("*"*80)
    print("The pixel size of Excalibur should be 55 um.")
    print(f"From calibration file, it is ({px_y*1000} um, {px_x*1000} um) (row, col)")
    print("The calibrated sample-to-detector distance might not be physical.")
    print("*"*80)
    
print(f"direct distance from sample to detector along the incident beam: {distance:.3f} mm")
print(f"pixel position of the beam center: ({bc_x/px_x:.3f}, {bc_y/px_y:.3f})")
print(f"tilt: {eub:.3f} deg")
print(f"Rotation of the tilt plan arround the Z-detector axis: {euc:.3f} deg")
print(f"Pixel size (x, y): ({px_x*1000:.3f}, {px_y*1000:.3f}) um")
print(f"Detector size (row, col): ({ydim}, {xdim})")
print(f"Wavelength: {wavelength:.3f} angstroms")

## Load flat field

In [ ]:
with h5py.File(flat_path, "r") as f:
    flat = f[flat_key][...]

## Create Detector

In [ ]:
detector = Detector(max_shape=(ydim, xdim))

## Create integrator

In [ ]:
ai = AzimuthalIntegrator(detector=detector, wavelength=wavelength*1e-10)

## Set parameters of the integrator

In [ ]:
ai.setFit2D(distance, bc_x/px_x, bc_y/px_y, tilt=eub, tiltPlanRotation=euc, 
            pixelX=px_x*1000, pixelY=px_y*1000, splineFile=None)

mask_crc = crc32(mask)
unit_pyfai = units.to_unit(unit_pyfai)
integr = ai.setup_sparse_integrator((ydim, xdim), npt_rad, mask, None, None, 
                                    mask_checksum=mask_crc, 
                                    unit=unit_pyfai, 
                                    split="bbox", algo=method[1],
                                    scale=False)

if radial_min is not None and radial_min<integr.pos0_min:
    radial_min = integr.pos0_min
elif radial_min is None:
    radial_min = integr.pos0_min

if radial_max is not None and radial_max>integr.pos0_max:
    radial_max = integr.pos0_max
elif radial_max is None:
    radial_max = integr.pos0_max

if azimuthal_min is None or azimuthal_min < -180:
    azimuthal_min = -180
if azimuthal_max is None or azimuthal_max > 180:
    azimuthal_max = 180

In [ ]:
# info passed to the integrator
kwargs_1d = {"npt": npt_rad,
             "method": method,
             "mask": mask,
             "azimuth_range": (azimuthal_min, azimuthal_max),
             "flat": flat,
             "unit": unit_pyfai,
             "correctSolidAngle": correctSolidAngle,
             "polarization_factor": polarization_factor,
             "radial_range": (radial_min, radial_max),
             "safe": safe,
             }

## Azimuthal integration will start here
### Initialise the instance responsible for the integration

In [ ]:
exi = ExcaliburXRDIntegration(inpath, 
                              outpath, 
                              ai,
                              regrid=do_regrid,
                              kwargs_1d=kwargs_1d,
                              crop_y=crop_y, 
                              crop_x=crop_x,
                              tth=tth, 
                              angstrom=angstrom, 
                             )

### Populate the instance's attributes with metadata

In [ ]:
exi.populate_metadata()

### Create the output file
This includes all axis information and other metadata.

In [ ]:
exi.create_output(process_configfile=process_configfile,
                  calibration_path=calibration_path,
                 )

### Prepare background options

In [ ]:
if remove_background:
    bg_opts = {"background_type": background_type,
               "polynomial_order": polynomial_order,
               "bg_start": bg_start,
               "bg_end": bg_end,
               "smoothing_weight": smoothing_weight,
              }
else:
    bg_opts = None

### Perform file I/O operation and azimuthal integration
This includes reading the detector data, azimuthal integration (with background removal/smoothing if enabled) and writing the spectrum to the output file.

In [ ]:
exi.read_integr_write(bg_opts=bg_opts)
exi.time_summary()

### Visualise azimuthal integrated data

In [ ]:
xrd00 = exi.first_spectrum
radial = exi.radial_axis

fig, ax = plt.subplots(figsize=(16,9))
ax.plot(radial, xrd00, "b-", label="raw")

if remove_background:
    xrd00_br = exi.first_spectrum_bg
    ax.plot(radial, xrd00_br, "r-", label="background removed")

ax.set_xlabel(unit)
ax.set_ylabel("Intensity")
ax.legend()
plt.show()